In [1]:
from utils.configuration import get_config_from_json
from utils.training_utilities import set_GPU
from seq2point.seq2point import SEQ2POINT
import builtins


builtins.GENERAL_CONFIG = get_config_from_json(description="General Settings", config_file="configs/general_config.json")
builtins.MODEL_CONFIG = get_config_from_json(description="Model Parameters", config_file="configs/model_config.json")
builtins.TRAINING_CONFIG = get_config_from_json(description="Training Configuration", config_file="configs/training_config.json")

In [2]:
GENERAL_CONFIG

{'DATA_PATH': '../data/',
 'PRE_TRAINED_MODEL_FLAG': False,
 'SAVE_PATH': 'models/',
 'LOAD_MODEL': '',
 'DESCRIPTION': 'General Settings'}

In [ ]:
data = pd.read_csv(GENERAL_CONFIG['DATA_PATH'])

In [3]:
model = SEQ2POINT().to(set_GPU())

Followings are the Model Parameters of your experiment..
{'CONV_KERNEL': [10, 8, 6, 5, 5],
 'CONV_LAYERS': 5,
 'CONV_PADDING': [0],
 'CONV_STRIDE': [1],
 'DESCRIPTION': 'Model Parameters',
 'INPUT_CHANNELS': [1, 30, 30, 40, 50],
 'LEFT_PAD': [4, 3, 2, 2, 2],
 'OUTPUT_CHANNELS': [30, 30, 40, 50, 50],
 'POOL_KERNEL': [],
 'POOL_STRIDE': [],
 'RIGHT_PAD': [5, 4, 3, 2, 2],
 'SEQUENCE_LENGTH': 599}

Initializing SEQ2POINT model archiecture

SEQ2POINT model archiecture has been initialized


In [7]:
train_loss, validation_loss = model.run()

Followings are the Training Configuration of your experiment..
{'DESCRIPTION': 'Training Configuration',
 'EARLY_STOPPING_THRESHOLD': 3,
 'LEARNING_RATE': 0.001,
 'LOSS': 'nn.MSELoss',
 'LOSS_REDUCTION': 'mean',
 'NUM_EPOCHS': 10,
 'OPTIMIZER': 'optim.Adam',
 'TRAIN_BATCH_SIZE': 64}

Summary of the model architecture
Error occured in forward method due to  mat1 and mat2 shapes cannot be multiplied (60x599 and 29950x1024)
Layer (type:depth-idx)                   Output Shape              Param #
├─Sequential: 1-1                        []                        --
|    └─ConstantPad1d: 2-1                [-1, 1, 608]              --
|    └─Conv1d: 2-2                       [-1, 30, 599]             330
|    └─ReLU: 2-3                         [-1, 30, 599]             --
|    └─ConstantPad1d: 2-4                [-1, 30, 606]             --
|    └─Conv1d: 2-5                       [-1, 30, 599]             7,230
|    └─ReLU: 2-6                         [-1, 30, 599]             --
|    └

In [ ]:
! conda install matplotlib

In [8]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.DataFrame({'Training Loss':train_loss, 'Validation Loss':validation_loss})
df.plot(linewidth=4, alpha=0.7, figsize=(12,7), label='Loss')
plt.xlim([0,10])
# plt.ylim(-20,100)
plt.title('Training Loss vs Validation Loss per Epoch', fontsize=20)
plt.grid(axis='y', alpha=.5)
plt.yticks(fontsize=12, alpha=.7)
plt.xticks(fontsize=12, alpha=.7)
plt.xlabel('Epoch', fontsize=18, alpha=.7)
plt.ylabel('Loss Value', fontsize=18, alpha=.7)
# Lighten borders
plt.gca().spines["top"].set_alpha(.0)
plt.gca().spines["bottom"].set_alpha(.3)
plt.gca().spines["right"].set_alpha(.0)
plt.gca().spines["left"].set_alpha(.3)

plt.legend(loc='upper center')
plt.show()

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
fsize = 11
tsize = 6
tdir = 'in'
major = 5.0
minor = 3.0
style = 'default'
plt.style.use(style)
plt.rcParams['legend.handlelength'] = 5.0
plt.rcParams['text.usetex'] = False
plt.rcParams['font.size'] = fsize
plt.rcParams['legend.fontsize'] = tsize
plt.rcParams['xtick.direction'] = tdir
plt.rcParams['ytick.direction'] = tdir
plt.rcParams['xtick.major.size'] = major
plt.rcParams['xtick.minor.size'] = minor
plt.rcParams['ytick.major.size'] = major
plt.rcParams['ytick.minor.size'] = minor
xsize = 16
ysize = 8
ymin=-1
ymax=1024
marker = 'o'
color='orange'
x_ticks = [f"House {i}" for i in house_reports.keys()]
x_ticks.append('All Houses')


fig, ax = plt.subplots(figsize=(xsize, ysize))
ax.plot(median_list, marker=marker, color=color)

ax.set_title('Median Values of Kettle Durations Per Household')
ax.set_ylabel('Duration in minutes')

ticks_loc = np.arange(0,len(median_list))
ax.xaxis.set_major_locator(mticker.FixedLocator(ticks_loc))
ax.set_xticklabels([x_ticks[x] for x in ticks_loc])
ax.yaxis.set_major_formatter(FormatStrFormatter('%.2f'))
# ax.set_yscale('log', basey=2)

ax.grid(True)
# ax.set_ylim(ymin, ymax)
plt.legend(loc='upper center')
fig.tight_layout()
plt.show()

In [ ]:
import torch.nn as nn

In [ ]:
string = "nn.MSELoss"

In [ ]:
globals()[torch]

In [ ]:
c = nn.MSELoss()

In [ ]:
eval(string)

In [ ]:
class Foo:
    pass

str_to_class("nn.MSE()")